In [1]:
import mmcv
import mmrotate
import mmdet
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
# Check mmcv installation
print(mmcv.__version__) #noqa E1101
# Check MMRotate installation
print(mmrotate.__version__)
print(mmrotate.__file__)
# Check MMDetection installation
print(mmdet.__version__)

print(get_compiling_cuda_version())
print(get_compiler_version())

2.0.0
1.0.0rc1
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/__init__.py
3.1.0
11.1
GCC 7.3


In [2]:
import mmengine
print(mmengine.__version__)

0.10.3


In [3]:
from mmengine.config import Config
from mmengine.runner import Runner

from mmdet.utils import register_all_modules as register_all_modules_mmdet
from mmdet.apis import inference_detector, init_detector

from mmrotate.registry import VISUALIZERS
from mmrotate.utils import register_all_modules

In [ ]:
from mmrotate.registry import MODELS
from mmrotate.testing import get_detector_cfg
#from torchinfo import summary

In [5]:
# register all modules in mmdet into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules_mmdet(init_default_scope=False)
register_all_modules(init_default_scope=False)

# Choose to use a config and initialize the detector
config = '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/RS_Tasks_Finetune/Rotated_Detection/mmrotate1.x/configs/mtp/dior-r/oriented_rcnn_rvsa_l_800_mae_mtp_diorr.py'
# Setup a checkpoint file to load
checkpoint = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'
# Set work_dir
work_dir = '/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/work_dir'
# Set the device to be used for evaluation
device='cuda:0'
# Set inference path
img = './07.jpg'

cfg = Config.fromfile(config)
cfg.load_from = checkpoint
cfg.work_dir = work_dir

# register all modules in mmrotate into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
# model = init_detector(cfg, checkpoint, palette="dota", device=device)
# model

In [6]:
from mmrotate.registry import DATASETS
from mmrotate.datasets.dota import DOTADataset

@DATASETS.register_module()
class ShipDataset(DOTADataset):
    """SAR ship dataset for detection."""
    
    METAINFO = {
            'classes':('ship',),
                # palette is a list of color tuples, which is used for visualization.
            'palette': [(165, 42, 42),]
        }

In [7]:
cfg.dataset_type = "ShipDataset"

cfg.train_cfg.val_interval = 3
cfg.val_evaluator.metric = 'mAP'
cfg.val_evaluator.interval = 3
cfg.default_hooks.checkpoint.interval = 1
# cfg.train_cfg.max_epochs
cfg.device = 'cuda:0'

In [8]:
cfg

Config (path: /mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/RS_Tasks_Finetune/Rotated_Detection/mmrotate1.x/configs/mtp/dior-r/oriented_rcnn_rvsa_l_800_mae_mtp_diorr.py): {'default_scope': 'mmrotate', 'default_hooks': {'timer': {'type': 'IterTimerHook'}, 'logger': {'type': 'LoggerHook', 'interval': 50}, 'param_scheduler': {'type': 'ParamSchedulerHook'}, 'checkpoint': {'type': 'CheckpointHook', 'interval': 1}, 'sampler_seed': {'type': 'DistSamplerSeedHook'}, 'visualization': {'type': 'mmdet.DetVisualizationHook'}}, 'env_cfg': {'cudnn_benchmark': False, 'mp_cfg': {'mp_start_method': 'fork', 'opencv_num_threads': 0}, 'dist_cfg': {'backend': 'nccl'}}, 'vis_backends': [{'type': 'LocalVisBackend'}], 'visualizer': {'type': 'RotLocalVisualizer', 'vis_backends': [{'type': 'LocalVisBackend'}], 'name': 'visualizer'}, 'log_processor': {'type': 'LogProcessor', 'window_size': 50, 'by_epoch': True}, 'log_level': 'INFO', 'load_from': '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa

In [9]:
cfg.train_dataloader.dataset.type  = cfg.dataset_type
print(cfg.train_dataloader.dataset.type)

cfg.train_dataloader.dataset.ann_file = "anns/"
#cfg.train_dataloader.dataset.img_prefix = 'images_png'
cfg.train_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
cfg.train_dataloader.dataset.data_root = "/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format"
del cfg['train_dataloader']['dataset']['ignore_keys']
del cfg['train_dataloader']['dataset']['datasets']


# del cfg['optim_wrapper']['optimizer']
del cfg['optim_wrapper']['constructor']
# del cfg['optim_wrapper']['paramwise_cfg']
# cfg.pop('optim_wrapper')
# cfg['optimizer']=dict(
#     betas=(
#         0.9,
#         0.999,
#     ), lr=0.0001, type='AdamW', weight_decay=0.05),

ShipDataset


In [10]:
# del cfg['test_dataloader']
# del cfg['val_dataloader']
cfg.test_dataloader.dataset.type  = cfg.dataset_type
cfg.test_dataloader.dataset.ann_file = "anns/"
#cfg.train_dataloader.dataset.img_prefix = 'images_png'
cfg.test_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
cfg.test_dataloader.dataset.data_root = "/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format"


cfg.val_dataloader.dataset.type  = cfg.dataset_type
cfg.val_dataloader.dataset.ann_file = "anns/"
#cfg.train_dataloader.dataset.img_prefix = 'images_png'
cfg.val_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
cfg.val_dataloader.dataset.data_root = "/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format"

In [11]:
cfg.checkpoint = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'

In [12]:
cfg['model']['backbone']['pretrained'] = None

In [13]:
# del cfg['pretrained']

In [14]:
cfg.pop('backend_args')

In [15]:
del cfg['test_dataloader']['dataset']['pipeline'][0]['backend_args']

In [16]:
del cfg['test_dataloader']['dataset']['backend_args']

In [17]:
del cfg['test_pipeline'][0]['backend_args']

In [18]:
del cfg['train_pipeline'][0]['backend_args']

In [19]:
del cfg['val_dataloader']['dataset']['backend_args']

In [20]:
del cfg['val_dataloader']['dataset']['pipeline'][0]['backend_args']

In [21]:
del cfg['val_pipeline'][0]['backend_args']

In [22]:
del cfg['val_evaluator']['interval']

In [23]:
cfg.dump("./ship_fine_tunning_v3.py")

# Train a detector

In [24]:
# register all modules in mmdet into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules_mmdet(init_default_scope=False)
register_all_modules(init_default_scope=False)

# register all modules in mmrotate into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
model = init_detector(cfg, cfg.checkpoint, palette="dota", device=cfg.device)


runner = Runner.from_cfg(cfg)
runner.train()

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
Loads checkpoint by local backend from path: /mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth
04/23 18:16:25 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.17 (default, Jul  5 2023, 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11                                                                                             │
│   12                                                                                             │
│   13 runner = Runner.from_cfg(cfg)                                                               │
│ ❱ 14 runner.train()                                                                              │
│   15                                                                                             │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmengine/runner/runner.py:1777 in    │
│ train                                                                                            │
│                                                                                                  │
│   1774 │   │   # This must be called **AFTER** model has been wrapped.                           │
│   1775 │   │   self._maybe_compile('train_step')                                                 │
│   1776 │   │                                                                                     │
│ ❱ 1777 │   │   model = self.train_loop.run()  # type: ignore                                     │
│   1778 │   │   self.call_hook('after_run')                                                       │
│   1779 │   │   return model                                                                      │
│   1780                                                                                           │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmengine/runner/loops.py:96 in run   │
│                                                                                                  │
│    93 │   │   self.runner.call_hook('before_train')                                              │
│    94 │   │                                                                                      │
│    95 │   │   while self._epoch < self._max_epochs and not self.stop_training:                   │
│ ❱  96 │   │   │   self.run_epoch()                                                               │
│    97 │   │   │                                                                                  │
│    98 │   │   │   self._decide_current_val_interval()                                            │
│    99 │   │   │   if (self.runner.val_loop is not None                                           │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmengine/runner/loops.py:112 in      │
│ run_epoch                                                                                        │
│                                                                                                  │
│   109 │   │   self.runner.call_hook('before_train_epoch')                                        │
│   110 │   │   self.runner.model.train()                                                          │
│   111 │   │   for idx, data_batch in enumerate(self.dataloader):                                 │
│ ❱ 112 │   │   │   self.run_iter(idx, data_batch)                                                 │
│   113 │   │                                                                                      │
│   114 │   │   self.runner.call_hook('after_train_epoch')                                         │
│   115 │   │   self._epoch += 1                             

In [ ]:
import os

cfg.dump("./ship_fine_tunning_v1.py")